# Assignment 5



# Write your name
Please write names below

Maxon Yin
my2883





In this assignment we will explore different portfolio allocation rules. We apply these approaches to "alpha" type strategies or to combine different systemtic factors. We will use there the 5 factor model as a laboratory to think about these allocation rules.

The five factor model includes 5 excess return portfolios

- Market minus risk-free rate
- Small minus big (go long small caps, short large caps)
- High minus Low (go long High Book-to-market stonks and short low Book to market stonks--also called value minus growth)
- Robust minus Weak ( go long high profitability firms short low profitability firms)
- Conservative minus aggressive Investment ( go long low investment firms short high investment firms)

# Exercises

**Exercise 1**

Let's start by importing our data.

We will import monthly data on the five factor model.

You should go to Ken French website and figure out the name of the data set.

An alternative way is to use

`from pandas_datareader.famafrench import get_available_datasets` (you need to !pip install pandas-datareader first!)

And call get_available_datasets to see the name of the different data set available (hint: try `datasets = get_available_datasets()` to get different data set names)

TIP: When you complete and run the code below, the data set should have 7 columns: Date, factors and the risk-free rate.

Also note that there is a lot of stuff on a given file, i.e., `ds` below will contain multiple datasets associated with this dataset you choose. (Hint: check the data type of `ds`).

You have to choose the right one--the one that contains monthly return data!

Note that the data is percent, so a 5% return shows us as 5 and not 0.05. While we will not cumulate returns here, it is important to keep this in mind.

Finally make sure the index is in the datetime format

Here is the code to get you started!

What would you replace `_` with in `web.DataReader(_, 'famafrench')`? Recall what we had in `datasets`.

```
import ___ as pd

import pandas_datareader.data as web
ds = web.DataReader(_, 'famafrench')
df=ds[_]
df.____ = pd.to_datetime(df.____.to_timestamp())




In [9]:
import pandas as pd
import pandas_datareader.data as web

ds = web.DataReader('F-F_Research_Data_5_Factors_2x3', 'famafrench')

df = ds[0]

df.index = pd.to_datetime(df.index.to_timestamp())


/tmp/ipython-input-3372196211.py:4: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ds = web.DataReader('F-F_Research_Data_5_Factors_2x3', 'famafrench')
/tmp/ipython-input-3372196211.py:4: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ds = web.DataReader('F-F_Research_Data_5_Factors_2x3', 'famafrench')


**Exercise 2**

Let's start by computing the mean-variance efficient portfolio across these five factors.

I would like you start by using the full sample to estimate the weights and the leverage up the weights so this portfolio has the same in sample volatility as the market portfolio.

Report weights and  the Sharpe ratio of the MVE portfolio.

Hint: take a look at your Assignment 3.

In [10]:
import numpy as np

factors = df[['Mkt-RF','SMB','HML','RMW','CMA']] / 100

mu = factors.mean()
Sigma = factors.cov()

Sigma_inv = np.linalg.inv(Sigma)
w_mve_raw = Sigma_inv @ mu
w_mve_raw = w_mve_raw / (np.ones(len(mu)) @ w_mve_raw)

sigma_mkt = factors['Mkt-RF'].std()
sigma_mve = np.sqrt(w_mve_raw.T @ Sigma.values @ w_mve_raw)
k = sigma_mkt / sigma_mve
w_mve = w_mve_raw * k

mu_p = w_mve @ mu
sharpe_mve = mu_p / sigma_mkt

w_mve, sharpe_mve


(array([ 0.855385  , -0.64402225,  0.83284928,  0.55492016, -0.35210052]),
 np.float64(0.38724167678600574))

**Exercise 4**


Now lets do the same thing for four additional portfolio allocation rules


1. Use an Equal weight approach to invest across these factors.
2. Risk Parity 1: Assume expected returns of each factor are of the form $E[R^i_t]=c*\sigma(R^i_t)$ so they are exactly proportional to their standard deviation. Meaning that instead of estimating expected excess returns you simply use the vector of standard deviations $c[\sigma(R^1_t),\sigma(R^2_t),\sigma(R^3_t),..]$. Apply the MVE formula given the estimated covariance matrix and these "estimates" for expected returns
3. Risk Parity 2: Also Assume that all factors have zero correlation with each other
4. Minimum-Variance investing: Assume expected excess return are the same across assets, for example $E[R^i_t]=1$



In [11]:
import numpy as np
import pandas as pd

factors = df[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']] / 100
mu = factors.mean()
Sigma = factors.cov()
sigma_mkt = factors['Mkt-RF'].std()
N = len(mu)

def scale_to_market_vol(w_raw, Sigma, sigma_mkt, mu):
    Sigma_val = Sigma.values
    sigma_p = np.sqrt(w_raw.T @ Sigma_val @ w_raw)
    k = sigma_mkt / sigma_p
    w = w_raw * k
    mu_p = w @ mu.values
    sharpe = mu_p / sigma_mkt
    return w, sharpe

weights_dict = {}
sharpe_dict = {}

w_eq_raw = np.ones(N) / N
w_eq, sharpe_eq = scale_to_market_vol(w_eq_raw, Sigma, sigma_mkt, mu)
weights_dict['Equal Weight'] = w_eq
sharpe_dict['Equal Weight'] = sharpe_eq

sigma_vec = factors.std()
mu_rp1 = sigma_vec

Sigma_inv = np.linalg.inv(Sigma.values)
w_rp1_raw = Sigma_inv @ mu_rp1.values
w_rp1_raw = w_rp1_raw / w_rp1_raw.sum()

w_rp1, sharpe_rp1 = scale_to_market_vol(w_rp1_raw, Sigma, sigma_mkt, mu)
weights_dict['Risk Parity 1'] = w_rp1
sharpe_dict['Risk Parity 1'] = sharpe_rp1

w_rp2_raw = 1 / sigma_vec.values
w_rp2_raw = w_rp2_raw / w_rp2_raw.sum()

w_rp2, sharpe_rp2 = scale_to_market_vol(w_rp2_raw, Sigma, sigma_mkt, mu)
weights_dict['Risk Parity 2'] = w_rp2
sharpe_dict['Risk Parity 2'] = sharpe_rp2

ones = np.ones(N)
w_mv_raw = Sigma_inv @ ones
w_mv_raw = w_mv_raw / w_mv_raw.sum()

w_mv, sharpe_mv = scale_to_market_vol(w_mv_raw, Sigma, sigma_mkt, mu)
weights_dict['Min-Var'] = w_mv
sharpe_dict['Min-Var'] = sharpe_mv

factor_names = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']

weights_df = pd.DataFrame(weights_dict, index=factor_names)
sharpe_df = pd.DataFrame.from_dict(sharpe_dict, orient='index', columns=['Sharpe'])

print("=== Weights (scaled to market volatility) ===")
display(weights_df)

print("\n=== Sharpe Ratios (using market vol) ===")
display(sharpe_df)


=== Weights (scaled to market volatility) ===


,Equal Weight,Risk Parity 1,Risk Parity 2,Min-Var
Mkt-RF,0.504477,0.452080,0.391562,0.245215
SMB,0.504477,0.901362,0.555984,1.134123
HML,0.504477,-0.056431,0.436043,-0.439198
RMW,0.504477,1.272130,0.693492,1.499401
CMA,0.504477,0.646072,0.611859,0.911978



=== Sharpe Ratios (using market vol) ===


,Sharpe
Equal Weight,0.241087
Risk Parity 1,0.221081
Risk Parity 2,0.225971
Min-Var,0.144085


**Exercise 5**

We now should have 5 strategies (MVE plus the 4 above). Compute the full sample Sharpe Ratio of all five trading strategies. Also compute the one of the market portfolio as a comparison.

Discuss what we can learn and not learn from such exercise. What are the issues with interpreting the differences in the full sample performance?

In [12]:
market_returns = factors['Mkt-RF']
market_sharpe = market_returns.mean() / market_returns.std()

strategy_returns = {}

for name, w in weights_dict.items():
    ret = factors.values @ w
    strategy_returns[name] = ret

strategy_returns_df = pd.DataFrame(strategy_returns, index=factors.index)

sharpe_results = strategy_returns_df.mean() / strategy_returns_df.std()
sharpe_results['Market'] = market_sharpe

print("=== Sharpe Ratios of Strategies (Full Sample) ===")
display(sharpe_results.to_frame("Sharpe Ratio"))

=== Sharpe Ratios of Strategies (Full Sample) ===


,Sharpe Ratio
Equal Weight,0.241087
Risk Parity 1,0.221081
Risk Parity 2,0.225971
Min-Var,0.144085
Market,0.242803


Comparing full-sample Sharpe ratios shows that the equal-weight factor portfolio performs almost identically to the market, while risk-parity variants and especially minimum-variance perform worse. However, these differences should not be overinterpreted. All strategies except equal-weight rely on full-sample estimates of means and covariances, which introduces severe look-ahead bias and estimation error. As a result, the portfolios are implicitly allowed to “see the future,” making their Sharpe ratios overly optimistic or distorted. Therefore, while the results describe historical behavior, they do not provide reliable guidance for out-of-sample performance.

**Exercise 6**

Lets pursue a rolling approach to evaluate these strategies.

I would like you to split the sample down the middle--how to find the middle of the sample?

And then proceed by estimating in the first half, then looking at the performance in the following month.

Then adding this month to the estimation sample, and so on.

I provide code below that constructs this for the standard MVE strategy.

I advise you to



```
import numpy as np
Rp=pd.DataFrame([],index=[],columns=[],dtype=float) # Create an empty dataframe
Re=__ # Dataframe that holds monthly returns for factors
start_date=__ # Compute the start date of performance evaluation
for date in Re[start_date:].index:
    # I am using DateOffset function to tell python to stop one month before
    # so the estimation sample will be all months up to one month before the return that I
    # I will be trading
    # this guarantees the strategy is VALID with respect to information
    ERe=Re[:date- pd.DateOffset(months=1)].mean()
    CovRe=Re[:date- pd.DateOffset(months=1)].cov()
    Target_vol=___ # target volatiility so that it matches market portfolio volatility as in Exercise 3

    X=np.linalg.inv(CovRe) @ ERe* (Targetvol/(ERe @ np.linalg.inv(CovRe) @ ERe)**0.5) # Can you see why we are doing this?
    #store the strategy returns of the relevant month
    Rp.at[date,'MVE']=X @ Re.loc[date]
```

The above is the code for MVE strategy. You should write similar codes for the rest of the four strategies in Exercise 4.

In [13]:
Rp = pd.DataFrame([], index=[], columns=[], dtype=float)

Re = factors
mid_point = len(Re) // 2
start_date = Re.index[mid_point]

for date in Re[start_date:].index:
    ERe = Re[:date - pd.DateOffset(months=1)].mean()
    CovRe = Re[:date - pd.DateOffset(months=1)].cov()
    Target_vol = Re['Mkt-RF'].std()

    X = np.linalg.inv(CovRe) @ ERe * (Target_vol / (ERe @ np.linalg.inv(CovRe) @ ERe)**0.5)
    Rp.at[date, 'MVE'] = X @ Re.loc[date]

    w_eq = np.ones(len(ERe)) / len(ERe)
    sigma_eq = (w_eq.T @ CovRe.values @ w_eq)**0.5
    k_eq = Target_vol / sigma_eq
    Rp.at[date, 'Equal'] = (w_eq * k_eq) @ Re.loc[date]

    sigma_vec = Re[:date - pd.DateOffset(months=1)].std()
    mu_rp1 = sigma_vec
    X_rp1 = np.linalg.inv(CovRe) @ mu_rp1.values
    X_rp1 = X_rp1 / X_rp1.sum()
    sigma_rp1 = (X_rp1.T @ CovRe.values @ X_rp1)**0.5
    k_rp1 = Target_vol / sigma_rp1
    Rp.at[date, 'RP1'] = (X_rp1 * k_rp1) @ Re.loc[date]

    w_rp2 = 1 / sigma_vec.values
    w_rp2 = w_rp2 / w_rp2.sum()
    sigma_rp2 = (w_rp2.T @ CovRe.values @ w_rp2)**0.5
    k_rp2 = Target_vol / sigma_rp2
    Rp.at[date, 'RP2'] = (w_rp2 * k_rp2) @ Re.loc[date]

    ones = np.ones(len(ERe))
    w_mv = np.linalg.inv(CovRe.values) @ ones
    w_mv = w_mv / w_mv.sum()
    sigma_mv = (w_mv.T @ CovRe.values @ w_mv)**0.5
    k_mv = Target_vol / sigma_mv
    Rp.at[date, 'MinVar'] = (w_mv * k_mv) @ Re.loc[date]


**Exercise 7**

Compute their Sharpe Ratios, volatilities, and expected returns

Is there a conclusion? What statistical test can you to see if the differences in expected returns is statistically significant?

Would that be enough to know that one of them is better strategy?  What source of difference of expected returns might not be so interesting for an investor

In [14]:
expected_returns = Rp.mean()
volatilities = Rp.std()
sharpes = expected_returns / volatilities

performance = pd.DataFrame({
    'Mean Return': expected_returns,
    'Volatility': volatilities,
    'Sharpe Ratio': sharpes
})
performance


,Mean Return,Volatility,Sharpe Ratio
MVE,0.002205,0.029683,0.074280
Equal,0.000704,0.037376,0.018848
RP1,-0.006864,0.043898,-0.156369
RP2,-0.001937,0.037844,-0.051175
MinVar,-0.012552,0.047177,-0.266055


Using the rolling out-of-sample procedure, the MVE strategy delivers the highest mean return (0.22% per month) and the highest Sharpe ratio (0.074). All other strategies perform substantially worse, with Equal Weight close to zero Sharpe (0.019) and the remaining three strategies showing negative Sharpe ratios. These results suggest that once estimation is restricted to information available at each point in time, the performance of most rules declines sharply relative to their full-sample Sharpe ratios. In other words, the historical advantages observed earlier do not survive an out-of-sample evaluation.<br>
However, the differences in mean returns across the strategies should not be over-interpreted. To formally test whether one strategy earns a significantly higher expected return than another, we could apply a Newey–West adjusted t-test to the series of return differences, or use a Diebold–Mariano test. But even statistical significance would not be enough to declare one strategy “better.” Return differences may arise from sampling noise, factor-cycle timing, leverage effects, or other sources that are not meaningful to investors. Thus, while the rolling exercise reveals substantial degradation in performance, it does not allow us to conclude that any strategy is reliably superior going forward.